<a href="https://colab.research.google.com/github/dqminhv/owid-copd-us-uganda/blob/main/us_uganda_copd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


To calculate both the crude death rate and the age-standardized death rate for chronic obstructive pulmonary disease (COPD) for all ages in both the United States and Uganda for 2019, you would typically follow these steps:

1. Obtain age-specific death rates from COPD for both the United States and Uganda
in 2019. These data can usually be obtained from national health departments, statistical agencies, or international health organizations such as the World Health Organization (WHO).

2. Calculate the crude death rate for COPD for each country by summing up the number of deaths from COPD for all age groups and dividing by the total population of the country for the same year. The formula for crude death rate is:

where the result is typically expressed per 1,000 population.

3. Calculate the age-standardized death rate for COPD for each country. Age-standardization allows for the comparison of death rates between populations with different age distributions by adjusting for age differences. This is important because the risk of death from COPD can vary by age. The age-standardized death rate is usually expressed per 100,000 population.

# 1.Import packages and data





In [1]:
# Import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Create paths for each data files
population_data_path = '/content/drive/MyDrive/Colab Notebooks/data/WPP2022_Population1JanuaryByAge5GroupSex_Medium.csv'
age_specific_death_rates_of_copd_data_path = '/content/drive/MyDrive/Colab Notebooks/data/age-specific-death-rates-of-copd.csv'
who_age_std_data_path = '/content/drive/MyDrive/Colab Notebooks/data/WHO_std_pop_dist.csv'

In [3]:
#Read the files into dataframes
world_pop = pd.read_csv(population_data_path, low_memory=False)
age_specific_death_rates_of_copd = pd.read_csv(age_specific_death_rates_of_copd_data_path)
who_age_std = pd.read_csv(who_age_std_data_path)

In [4]:
#Create a data frame for US population in 2019 from the world population data
us_pop_2019 = world_pop.loc[(world_pop['Location']=='United States of America') & (world_pop['Time']==2019)][['ISO2_code', 'AgeGrp', 'PopTotal']].reset_index(drop=True)

In [5]:
#Create a data frame for Uganda population in 2019 from the world population data
uganda_pop_2019 = world_pop.loc[(world_pop['Location']=='Uganda') & (world_pop['Time']==2019)][['ISO2_code', 'AgeGrp', 'PopTotal']].reset_index(drop=True)

# 2.Grouping age groups from 85+ together

In the UN World Population data, ages above 85 are divided into smaller subgroups, whereas the WHO Standard Population and Table of age-specific death rates of COPD group all ages above 85 into a single category. Therefore, we must consolidate the age groups above 85 in the UN World Population data into a single group labeled 85+.

## a. United States

In [6]:
# 85+ row data by summing up all age groups above 85
us_85_pop = us_pop_2019['PopTotal'][-4:].sum()
us_85 = pd.DataFrame({'ISO2_code': ['US'], 'AgeGrp': ['85+'], 'PopTotal': [us_85_pop]})

# Add the new row
us_pop_2019 = pd.concat([us_pop_2019, us_85], ignore_index=True)

In [7]:
#Remove rows 17 to 20, which are the groups of 85-89, 90-94, 95-99, 100+
us_pop_2019.drop(index=[17, 18, 19, 20], inplace=True)
us_pop_2019.reset_index(drop=True, inplace=True)

## b.Uganda

In [8]:
# 85+ row data by summing up all age groups above 85
uganda_85_pop = uganda_pop_2019['PopTotal'][-4:].sum()
uganda_85 = pd.DataFrame({'ISO2_code': ['UG'], 'AgeGrp': ['85+'], 'PopTotal': [uganda_85_pop]})

# Add the new row
uganda_pop_2019 = pd.concat([uganda_pop_2019, uganda_85], ignore_index=True)

In [9]:
#Remove rows 17 to 20, which are the groups of 85-89, 90-94, 95-99, 100+
uganda_pop_2019.drop(index=[17, 18, 19, 20], inplace=True)
uganda_pop_2019.reset_index(drop=True, inplace=True)

# 3.Crude death rates by COPD

## a. United States

In [10]:
# Crude death rate = Total death / Total population
us_crude_death_rate_2019 = \
(us_pop_2019['PopTotal'] * (age_specific_death_rates_of_copd['Death rate, United States, 2019'])).sum() / us_pop_2019['PopTotal'].sum()

In [11]:
print('Crude death rate for COPD of the US in 2019 is {} per 100,000'.format(round(us_crude_death_rate_2019, 1)))

Crude death rate for COPD of the US in 2019 is 56.9 per 100,000


## b. Uganda

In [12]:
# Crude death rate = Total death / Total population
uganda_crude_death_rate_2019 = \
(uganda_pop_2019['PopTotal'] * (age_specific_death_rates_of_copd['Death rate, Uganda, 2019'])).sum() / uganda_pop_2019['PopTotal'].sum()

In [13]:
print('Crude death rate for COPD of Uganda in 2019 is {} per 100,000'.format(round(uganda_crude_death_rate_2019, 1)))

Crude death rate for COPD of Uganda in 2019 is 5.8 per 100,000


#4.Age-standardized death rate for COPD

## a. United States

In [14]:
# Calculate the standardized population of each age group in the US
# Formula: Standardized population of age group G = Total population * Age-standardized rate of group G
us_pop_age_std_2019 = who_age_std['who-std']/100 * (us_pop_2019['PopTotal'].sum())

In [15]:
# Age-standardize death rate = Total age-standardized death rate / Total population
us_age_std_death_rate_2019 = \
(us_pop_age_std_2019 * (age_specific_death_rates_of_copd['Death rate, United States, 2019'])).sum() / us_pop_2019['PopTotal'].sum()

In [16]:
# Print the result
print('Age-standardized death rate for COPD of the US in 2019 is {} per 100,000'.format(round(us_age_std_death_rate_2019, 1)))

Age-standardized death rate for COPD of the US in 2019 is 28.4 per 100,000


## b. Uganda

In [17]:
# Calculate the standardized population of each age group in Uganda
# Formula: Standardized population of age group G = Total population * Age-standardized rate of group G
uganda_pop_age_std_2019 = who_age_std['who-std']/100 * (uganda_pop_2019['PopTotal'].sum())

In [18]:
# Age-standardize death rate = Total age-standardized death rate / Total population
uganda_age_std_death_rate_2019 = \
(uganda_pop_age_std_2019 * (age_specific_death_rates_of_copd['Death rate, Uganda, 2019'])).sum() / uganda_pop_2019['PopTotal'].sum()

In [19]:
print('Age-standardized death rate for COPD of Uganda in 2019 is {} per 100,000'.format(round(uganda_age_std_death_rate_2019, 1)))

Age-standardized death rate for COPD of Uganda in 2019 is 28.7 per 100,000
